# Data 620 Project 3
## Classification
Jit Seneviratne and Sheryl Piechocki 
June 29, 2020

Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python,
and any features you can think of, build the best name gender classifier you can.
Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the devtest set, and the remaining 6900 words for the training set. Then, starting with the example name gender
classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are
satisfied with your classifier, check its final performance on the test set.
How does the performance on the test set compare to the performance on the dev-test set? Is this what
you'd expect? 

**Dataset**
The data used in this project is the names corpus included in the NLTK package.

**Analysis:**


In [90]:
from nltk.corpus import names
import random
from nltk.classify import apply_features
%matplotlib inline
import pandas as pd
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
import nltk, re, pprint
from nltk import word_tokenize
import string
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

### Investigate the NLTK Names Corpus

In [70]:
 names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])

In [71]:
#print(names[:10])
print('Count of total names in the corpus is: ' , (len(names)))

females = [(name, gender) for name, gender in names if gender == 'female']
print('Count of female names in the corpus is: ' , (len(females)))
males = [(name, gender) for name, gender in names if gender == 'male']
print('Count of male names in the corpus is: ' , (len(males)))

Count of total names in the corpus is:  7944
Count of female names in the corpus is:  5001
Count of male names in the corpus is:  2943


### Remove any trailing spaces from names

In [72]:
names = list(tuple("".join(i.rsplit()) for i in a) for a in names)

### Split the names data set into train, test, and devtest

In [73]:
random.shuffle(names)
train_names = names[1000:]
devtest_names = names[500:1000]
test_names = names[:500]

In [74]:
print(len(train_names))
print(len(devtest_names))
print(len(test_names))

6944
500
500


### Create dataframes for later use

In [77]:
train_name_df = pd.DataFrame(train_names)
train_name_df.columns = ['name', 'gender']
test_name_df = pd.DataFrame(test_names)
test_name_df.columns = ['name', 'gender']

### Last letter feature
Use the function provided in the book that takes an input word and returns the last letter

In [6]:
 def gender_features(name):
        return {'last_letter': name[-1]}

### Run 1 - Max Entropy Classifier  
Feature: last letter.


In [7]:
train_set = apply_features(gender_features, train_names)
test_set = apply_features(gender_features, test_names)
devtest_set = apply_features(gender_features, devtest_names)

#classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier = nltk.MaxentClassifier.train(train_set, algorithm='iis', trace=0, max_iter=1000)
print('Run 1: Gender correctly identified: ', "{:.1%}".format(nltk.classify.accuracy(classifier, test_set)))

In [8]:
print('Run 1: Gender correctly identified: ', "{:.1%}".format(nltk.classify.accuracy(classifier, test_set)))

Run 1: Gender correctly identified:  77.4%


### Run 1 - Most informative features  
Using the last letter feature, the classifier ranks names ending in 'c' as the most informative feature and is positive for males.  

In [9]:
classifier.show_most_informative_features(5)

   9.966 last_letter=='c' and label is 'male'
   9.966 last_letter=='p' and label is 'male'
  -4.824 last_letter=='a' and label is 'male'
  -3.392 last_letter=='k' and label is 'female'
  -2.644 last_letter=='f' and label is 'female'


### Add additional features for first letter and length of the name


In [10]:
def gender_features2(name):
    features = {}
    features["last_letter"] = name[-1].lower()
    features["first_letter"] = name[0].lower()
    features["name_length"] = len(name)
    return features


### Run 2 - Max Entropy Classifier  
Features: last letter, first letter, length of name

In [11]:
train_set2 = apply_features(gender_features2, train_names)
test_set2 = apply_features(gender_features2, test_names)
devtest_set2 = apply_features(gender_features2, devtest_names)

#classifier2 = nltk.NaiveBayesClassifier.train(train_set2)
classifier2 = nltk.MaxentClassifier.train(train_set2, algorithm='iis', trace=0, max_iter=1000)
print('Run 2: Gender correctly identified: ', "{:.1%}".format(nltk.classify.accuracy(classifier2, test_set2)))

Run 2: Gender correctly identified:  79.0%


### Run 2 - Create Confusion Matrix

In [61]:
tag2 = []
guess2 = []
for  (name, label) in test_names:
    observed2 = classifier2.classify(gender_features2(name))
    tag2.append(label)
    guess2.append(observed2)

print(nltk.ConfusionMatrix(tag2, guess2))

       |   f     |
       |   e     |
       |   m   m |
       |   a   a |
       |   l   l |
       |   e   e |
-------+---------+
female |<253> 53 |
  male |  52<142>|
-------+---------+
(row = reference; col = test)



### Run 2 - Check the errors

In [14]:
errors = []
for (name, tag) in devtest_names:
    guess =  classifier2.classify(gender_features2(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

for (tag, guess, name) in sorted(errors): 
    print('correct=%-8s guess=%-8s name=%-30s'  %
          (tag, guess, name))
print(len(errors))

correct=female   guess=male     name=Arlyn                         
correct=female   guess=male     name=Astrix                        
correct=female   guess=male     name=Bert                          
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Bo                            
correct=female   guess=male     name=Brandais                      
correct=female   guess=male     name=Charo                         
correct=female   guess=male     name=Cher                          
correct=female   guess=male     name=Consuelo                      
correct=female   guess=male     name=Deb                           
correct=female   guess=male     name=Demeter                       
correct=female   guess=male     name=Devin                         
correct=female   guess=male     name=Drew                          
correct=female   guess=male     name=Ester                         
correct=female   guess=male     name=Gabriell   

### Run 2 - Most Important Features

In [15]:
classifier2.show_most_informative_features(5)

   9.351 name_length==14 and label is 'female'
   8.777 name_length==13 and label is 'female'
   8.742 last_letter=='c' and label is 'male'
   8.513 last_letter=='p' and label is 'male'
  -5.240 last_letter=='a' and label is 'male'


### Add additional features - counts of "a", "i", "o", "y" and create dummy columns for first and last letter features

In [78]:
train_name_df['last_letter'] = train_name_df['name'].apply(lambda x: x[-1])
train_name_df['first_letter'] = train_name_df['name'].apply(lambda x: x[0])
train_name_df['len_name'] = train_name_df['name'].apply(lambda x: len(x))
train_name_df['a_count'] = train_name_df['name'].apply(lambda x: len(re.findall('a',x)))
train_name_df['i_count'] = train_name_df['name'].apply(lambda x: len(re.findall('i',x)))
train_name_df['o_count'] = train_name_df['name'].apply(lambda x: len(re.findall('o',x)))
train_name_df['y_count'] = train_name_df['name'].apply(lambda x: len(re.findall('y',x)))
train_name_df = pd.get_dummies(train_name_df, columns=['last_letter','first_letter'])

In [81]:
train_name_df.head()

,name,gender,len_name,a_count,i_count,o_count,y_count,last_letter_a,last_letter_b,last_letter_c,...,first_letter_Q,first_letter_R,first_letter_S,first_letter_T,first_letter_U,first_letter_V,first_letter_W,first_letter_X,first_letter_Y,first_letter_Z
0,Jesselyn,female,8,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Thor,male,4,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Nanice,female,6,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marwin,male,6,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angie,male,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
test_name_df['last_letter'] = test_name_df['name'].apply(lambda x: x[-1])
test_name_df['first_letter'] = test_name_df['name'].apply(lambda x: x[0])
test_name_df['len_name'] = test_name_df['name'].apply(lambda x: len(x))
test_name_df['a_count'] = test_name_df['name'].apply(lambda x: len(re.findall('a',x)))
test_name_df['i_count'] = test_name_df['name'].apply(lambda x: len(re.findall('i',x)))
test_name_df['o_count'] = test_name_df['name'].apply(lambda x: len(re.findall('o',x)))
test_name_df['y_count'] = test_name_df['name'].apply(lambda x: len(re.findall('y',x)))
test_name_df = pd.get_dummies(test_name_df, columns=['last_letter','first_letter'])

In [80]:
test_name_df = test_name_df.reindex(columns = train_name_df.columns, fill_value=0)

In [82]:
test_name_df.head()

,name,gender,len_name,a_count,i_count,o_count,y_count,last_letter_a,last_letter_b,last_letter_c,...,first_letter_Q,first_letter_R,first_letter_S,first_letter_T,first_letter_U,first_letter_V,first_letter_W,first_letter_X,first_letter_Y,first_letter_Z
0,Ramesh,male,6,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Rawley,male,6,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Maximilian,male,10,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Filmore,male,7,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Andi,female,4,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Run 3 - Logistic Classifier with additional features

In [83]:
def logistic_classifier(train_df, test_df):
    
    X_train = train_df.drop(labels=['gender','name'],
                axis=1)
        
    y_train = train_df['gender']
    
    X_test = test_df.drop(labels=['gender','name'],
                axis=1)
        
    y_test = test_df['gender']
    
    lr = LogisticRegression(penalty='l2',
                            dual=False,
                            max_iter=1000,
                            tol=.0001)
    
    pipeline = Pipeline(steps=[('logistic', lr)])
    
    param_grid = {
    'logistic__C': np.arange(1, 50, 10)}

    cv = ShuffleSplit(n_splits=10, test_size=.3, random_state=0)

    search = GridSearchCV(pipeline, 
                          param_grid,
                          cv=cv,
                          return_train_score=True
                          )

    search.fit(X_train,y_train)

    y_pred = search.predict(X_test)
    
    print('Run 3: Logistic Classifier')
    print('Train: Gender correctly identified: ', "{:.1%}".format(search.score(X_train, y_train)))
    print('Test: Gender correctly identified: ', "{:.1%}".format(search.score(X_test, y_test)))
    print('')
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))
    

In [84]:
logistic_classifier(train_name_df, test_name_df)

Run 3: Logistic Classifier
Train: Gender correctly identified:  78.7%
Test: Gender correctly identified:  79.8%

Confusion Matrix
[[285  35]
 [ 66 114]]


### Run 4 - Naive Bayes Classifier with additional features

In [85]:
def m_naive_bayes_classifier(train_df, test_df):
    
    X_train = train_df.drop(labels=['gender','name'],
                axis=1)
        
    y_train = train_df['gender']
    
    X_test = test_df.drop(labels=['gender','name'],
                axis=1)
        
    y_test = test_df['gender']
    
    nb = MultinomialNB()
    
    # Create a pipeline that standardizes, then runs logistic regression
    pipeline = Pipeline(steps=[('nb', nb)])
    
    param_grid = {
   'nb__alpha': np.arange(.1, 1, .2)}

    cv = ShuffleSplit(n_splits=10, test_size=.3)

    search = GridSearchCV(pipeline, 
                          param_grid,
                          cv=cv,
                          return_train_score=True
                          )

    search.fit(X_train,y_train)

    y_pred = search.predict(X_test)
    
    
    print('Run 4: Naive Bayes Classifier')
    print('Train: Gender correctly identified: ', "{:.1%}".format(search.score(X_train, y_train)))
    print('Test: Gender correctly identified: ', "{:.1%}".format(search.score(X_test, y_test)))
    print('')
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))

In [86]:
m_naive_bayes_classifier(train_name_df, test_name_df)

Run 4: Naive Bayes Classifier
Train: Gender correctly identified:  77.6%
Test: Gender correctly identified:  76.4%

Confusion Matrix
[[273  47]
 [ 71 109]]


### Run 5 - Random Forest Classifier with additional features

In [87]:
def random_forest_classifier(train_df, test_df):
    
    X_train = train_df.drop(labels=['gender','name'],
                axis=1)
        
    y_train = train_df['gender']
    
    X_test = test_df.drop(labels=['gender','name'],
                axis=1)
        
    y_test = test_df['gender']
    
    rf=RandomForestClassifier()
   

    # Create a pipeline that standardizes, then runs logistic regression
    pipeline = Pipeline(steps=[('rf', rf)])
    
    param_grid = {
    'rf__min_samples_split': np.arange(2, 10, 2),
    'rf__n_estimators': np.arange(10, 20, 5)}

    cv = ShuffleSplit(n_splits=10, test_size=.3)

    search = GridSearchCV(pipeline, 
                          param_grid,
                          cv=cv,
                          return_train_score=True
                          )

    search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    print('Run 5: Random Forest Classifier')
    print('Train: Gender correctly identified: ', "{:.1%}".format(search.score(X_train, y_train)))
    print('Test: Gender correctly identified: ', "{:.1%}".format(search.score(X_test, y_test)))
    print('')
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))

In [88]:
random_forest_classifier(train_name_df, test_name_df)

Run 5: Random Forest Classifier
Train: Gender correctly identified:  87.1%
Test: Gender correctly identified:  79.4%

Confusion Matrix
[[279  41]
 [ 62 118]]


In all cases, the minority class has been misclassified a little bit. In the future, we'll upsample the minority class.

### Upsample Minorty Class and Rerun

In [112]:
train_name_df['gender'].value_counts()

female    4357
male      2587
Name: gender, dtype: int64

In [108]:
df_minority = train_name_df[train_name_df['gender']=='male']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=2000,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([train_name_df, 
                          df_minority_upsampled])

In [109]:
logistic_classifier(df_upsampled, test_name_df)

Run 3: Logistic Classifier
Train: Gender correctly identified:  78.4%
Test: Gender correctly identified:  77.8%

Confusion Matrix
[[251  69]
 [ 42 138]]


In [110]:
m_naive_bayes_classifier(df_upsampled, test_name_df)

Run 4: Naive Bayes Classifier
Train: Gender correctly identified:  76.6%
Test: Gender correctly identified:  77.4%

Confusion Matrix
[[246  74]
 [ 39 141]]


In [111]:
random_forest_classifier(df_upsampled, test_name_df)

Run 5: Random Forest Classifier
Train: Gender correctly identified:  87.9%
Test: Gender correctly identified:  78.2%

Confusion Matrix
[[259  61]
 [ 48 132]]
